# Sesión 3 - Scrapy - Ejemplo Crawling JSON-LD (ElMundo)

## Apartado 1.1 Crawler de El Mundo extrayendo JSON-LD
En el ejemplo siguiente definimos un crawler en Scrapy para extraer noticias de los metadatos de las páginas web.
El Mundo y otros periódicos publican metadatos en formato JSON-LD (https://json-ld.org/) que permite obtener información estructurada de las webs. En este caso, tendremos que obtener esos objetos JSON-LD y extraer su información en el formato de noticia (https://schema.org/NewsArticle) publicado en Schema.org

** Existen librerías de Python para trabajar directamente con JSON-LD

In [ ]:
!pip3 install -U scrapy

In [2]:
import scrapy
import sys
import json
import locale
import time
import random
from bs4 import BeautifulSoup

class ElMundoSpider (scrapy.Spider):

    #Es obligatorio poner un nombre
    name = 'elmundo'

    #Ponemos que el dominio que está permitido es el de la página y no vamos a irnos fuera de la misma
    allowed_domains = ['www.elmundo.es']

    start_urls = ['https://www.elmundo.es/']

    # para evitar que el sitio te bloquee por usar scrapy es interesante cambiar el USER_AGENT
    # El user agent por defecto de Scrapy cuando hace una petición es
    # Scrapy/VERSION (+https://scrapy.org)
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    def parse (self, response):
      """
      @inherit

      @param self
      @param response
      """

      # Obtenemos la URL de la web que estamos procesando
      url = str(response.request.url).strip()

      # Obtenemos usando CSS la parte donde está definido el JSON-LD en la web
      for item in response.css('script[type=\"application/ld+json\"]'):
        data = json.loads(item.css('::text').get())
        # Procesamos aquí los datos
        # RELLENAR
        title = ""
        content = ""
        date = ""
        if url :

            print ("-------------------------")
            print (url)
            print (title)
            print (content)
            print (date)
            print ("-------------------------")

            data = {
                'url' : url,
                'title': title,
                'content': content,
                'date': date
            }

            #Creamos para cada item un fichero json y para ello obtenemos un número aleatorio.
            filename = str(random.random()).replace(".","") + ".json"

            # Guardamos el documento si tiene contenido y título
            if content and title:
                with open ('elmundo/' + filename, 'w') as f:
                    json.dump (data, f, indent = 4)

      # Por último, miramos distintos enlaces que existan en la web (en este caso todos los de la etiqueta <a>)
      url_in_current_document = response.css ('a')
      for next_page in url_in_current_document:
          # Por cada enlace ejecutamos el mismo parse
          yield response.follow (next_page, self.parse)


## Apartado 1.2
Para poder lanzar el Spider necesitamos que ejecutar el siguiente código donde se configuará y lanzará el proceso.
Hay que hacer notar que solamente se puede lanzar un proceso por cada sesión en Jupyter notebook es por eso por lo que se recomienda exportar el código en un script de Python .py para poder ejecutarlo desde la línea de comandos.

In [ ]:
import os
import scrapy
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "rss"
# Comprobamos que existe la carpeta y si no existe la creamos
if (not os.path.exists('elmundo')):
    os.mkdir('elmundo')

# Creamos el proceso con el RSSSpider
process.crawl(ElMundoSpider)
# Ejecutamos el Crawler
process.start()